# Inicialização

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy.spatial import distance
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
pasta = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Scilab\\Pêndulo\\Dados\\'
D = [None] * 4
D[0] = pd.read_csv(pasta+'DaPredv1.csv',header=None).values
D[1] = pd.read_csv(pasta+'DbPredv1.csv',header=None).values
D[2] = pd.read_csv(pasta+'DcPredv1.csv',header=None).values
D[3] = pd.read_csv(pasta+'DdPredv1.csv',header=None).values

In [4]:
for i in D:
    print(i.shape)

(1000, 1203)
(1000, 1203)
(1000, 1203)
(1000, 1203)


In [19]:
fig = px.line(D[0][15,:1000]/4)
fig.add_trace(go.Scatter(y=D[1][15,:1000], mode='lines', showlegend=True))
fig.add_trace(go.Scatter(y=D[2][15,:1000], mode='lines', showlegend=True))
fig.add_trace(go.Scatter(y=D[3][15,:1000], mode='lines', showlegend=True))

In [22]:
tarefas = np.array(['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking'])
SAC = np.array(['sitting', 'standing', 'walking', 'climbingup', 'climbingdown', 'running'])
posicao = np.array(['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist'])

In [23]:
pasta = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\realworld views\\'
D = [None] * 7
D[0] = pd.read_csv(pasta+posicao[0]+'_nova50hz.csv')
D[1] = pd.read_csv(pasta+posicao[1]+'_nova50hz.csv')
D[2] = pd.read_csv(pasta+posicao[2]+'_nova50hz.csv')
D[3] = pd.read_csv(pasta+posicao[3]+'_nova50hz.csv')
D[4] = pd.read_csv(pasta+posicao[4]+'_nova50hz.csv')
D[5] = pd.read_csv(pasta+posicao[5]+'_nova50hz.csv')
D[6] = pd.read_csv(pasta+posicao[6]+'_nova50hz.csv')

# Nova view realworld

In [ ]:
# Vamos definir a pasta que está o dataset
workspace = 'G:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\'
root = 'G:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\RealWorld\\'

# Lista de usuários e atividades
users = natsorted(os.listdir(root))
tarefas = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking']
SAC = ['sitting', 'standing', 'walking', 'climbingup', 'climbingdown', 'running']
posicao = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
posicaopt = ['peito', 'antebraço', 'cabeça', 'canela', 'coxa', 'braço', 'cintura']

In [ ]:
# Vamos nos preparar para criar as views
map = [4, 3, -1, -1, 5, 0, 1, 2]
h = signal.butter(3, 0.3, 'hp', fs=50, output='sos')

colunas = ['accel-x-' + str(x) for x in np.arange(150).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(150).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(150).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(150).tolist()]
colunas += ['activity code']
colunas += ['user']
colunas += ['standard activity code']

In [ ]:
# Vamos criar as views
for p in posicao:

    chunk = np.zeros((len(users)*len(tarefas)*600,150*6+3))
    cont = 0

    for usuario in tqdm(users):
        filesacc = sorted(os.listdir(workspace+'realworldcsvs\\'+usuario+'\\acc'))
        filesgyr = sorted(os.listdir(workspace+'realworldcsvs\\'+usuario+'\\gyr'))
        pos = []
        for i in range(len(filesacc)):
            if filesacc[i].find(p)>-1:
                pos.append(i)
        for i in pos:
            acc = pd.read_csv(workspace+'realworldcsvs\\'+usuario+'\\acc\\'+filesacc[i])
            gyr = pd.read_csv(workspace+'realworldcsvs\\'+usuario+'\\gyr\\'+filesgyr[i])
            for atividade in tarefas:
                if filesacc[i].find(atividade)>-1:
                    break
            fA = np.array(acc.iloc[:,2:]).transpose()
            fG = np.array(gyr.iloc[:,2:]).transpose()

            Na = acc.shape[0]
            Ng = gyr.shape[0]
            starta = acc['attr_time'][0]
            stopa =  acc['attr_time'].iloc[-1]
            ta = np.linspace(starta, stopa, num=Na)
            startg = gyr['attr_time'][0]
            stopg =  gyr['attr_time'].iloc[-1]
            tg = np.linspace(startg, stopg, num=Ng)            
            intp1 = interpolate.interp1d(ta, fA, kind='cubic')
            intp2 = interpolate.interp1d(tg, fG, kind='cubic')
            nt = np.arange(max(starta,startg), min(stopa,stopg), 20)

            fA = intp1(nt)
            fG = intp2(nt)

            fA = signal.sosfiltfilt(h, fA, axis=1)
            NJ = min(fA.shape[1],fG.shape[1])//150
            aux = np.zeros(150*6+3)
            for j in range(NJ):
                aux[:450] = fA[:,j*150:(j+1)*150].reshape(150*3)
                aux[450:900] = fG[:,j*150:(j+1)*150].reshape(150*3)
                aux[-1] = map[tarefas.index(atividade)]
                aux[-2] = users.index(usuario)
                aux[-3] = tarefas.index(atividade)
                chunk[cont,:] = aux
                cont += 1
    chunk = chunk[:cont,:]
    df = pd.DataFrame(data=chunk)
    df.columns = colunas
    df = df.astype({"activity code": int, "user": int, "standard activity code": int})
    df.to_csv(workspace+p+'_nova50hz.csv',index=False)

# Treinamento com dados do pêndulo

In [13]:
fig = px.line(D[0][16,:1000]/4)
fig.add_trace(go.Scatter(y=D[2][16,:1000], mode='lines', showlegend=False))

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            # 1st hidden layer
            nn.Flatten(start_dim=1),
            nn.Linear(300, 100),
            nn.Tanh(),

            # 2nd hidden layer
            nn.Linear(100, 2)
            # nn.Softmax(dim=1),

#             # output layer
#             nn.Linear(15, num_classes),
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits

In [26]:
class MLP2(nn.Module):
    def __init__(self):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            # 1st hidden layer
            nn.Flatten(start_dim=1),
            nn.Linear(24, 100),
            nn.Tanh(),

            # 2nd hidden layer
            nn.Linear(100, 2)
            # nn.Softmax(dim=1),

#             # output layer
#             nn.Linear(15, num_classes),
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits

In [5]:
class ChangEncoder2(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, device=device)
        self.relu1 = nn.LeakyReLU(0.3)
        self.inst1 = nn.InstanceNorm1d(16, device=device)
        self.drop1 = nn.Dropout(p=0.2)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, device=device)
        self.relu2 = nn.LeakyReLU(0.3)
        self.inst2 = nn.InstanceNorm1d(16, device=device)
        self.drop2 = nn.Dropout(p=0.2)

        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=4, device=device)
        self.relu3 = nn.LeakyReLU(0.3)
        self.inst3 = nn.InstanceNorm1d(32, device=device)
        self.drop3 = nn.Dropout(p=0.2)

        self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=1, device=device)
        self.relu4 = nn.LeakyReLU(0.3)
        self.inst4 = nn.InstanceNorm1d(32, device=device)
        self.drop4 = nn.Dropout(p=0.2)

        self.conv5 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=4, device=device)
        self.relu5 = nn.LeakyReLU(0.3)
        self.inst5 = nn.InstanceNorm1d(64, device=device)
        self.drop5 = nn.Dropout(p=0.2)

        self.conv6 = nn.Conv1d(in_channels=64, out_channels=100, kernel_size=5, stride=1, device=device)
        self.relu6 = nn.LeakyReLU(0.3)

        self.glap = nn.AvgPool1d(kernel_size=4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.inst1(x)
        x = self.drop1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.inst2(x)
        x = self.drop2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.inst3(x)
        x = self.drop3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.inst4(x)
        x = self.drop4(x)
        
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.inst5(x)
        x = self.drop5(x)
        
        x = self.conv6(x)
        x = self.relu6(x)

        x = self.glap(x)
        x = x.flatten(start_dim=1)

        logits = x
        return logits

In [6]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()

        self.densa = nn.Linear(in_features=100, out_features=2, device=device)

    def forward(self, x):
        logits = self.densa(x)
        return logits

In [7]:
def ajusteDados(X):
    N = X.shape[0]
    Nj = X.shape[2]
    S = np.zeros((N*(Nj-151), 2, 150))
    y = np.zeros((N*(Nj-151), 2))
    for i in range(N):
        for j in range(Nj-151):
            S[(i*(Nj-151)+j),:,:] = X[i,:,j:(j+150)]
            y[(i*(Nj-151)+j),:] = X[i,:,(j+150)]
    return S, y

In [41]:
def ajusteDados2(X):
    N = X.shape[0]
    Nj = X.shape[2]
    S = np.zeros((N*(Nj-13), 2, 12))
    y = np.zeros((N*(Nj-13), 2))
    for i in range(N):
        for j in range(Nj-13):
            S[(i*(Nj-13)+j),:,:] = X[i,:,j:(j+12)]
            y[(i*(Nj-13)+j),:] = X[i,:,(j+12)]
    return S, y

In [42]:
dados = D[3]
N = dados.shape[0]
X = dados[:,:1200].reshape(N,2,600) / 50   # Normalizando para a faixa de -1 a 1 aprox.
y = dados[:,-1]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

X_train, s_train = ajusteDados2(X_train)
X_val, s_val = ajusteDados2(X_val)
X_test, s_test = ajusteDados2(X_test)

In [43]:
X_val.shape

(46960, 2, 12)

In [46]:
def treinoModelo2(dados):
    N = dados.shape[0]
    X = dados[:,:1200].reshape(N,2,600) / 50   # Normalizando para a faixa de -1 a 1 aprox.
    y = dados[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
    
    X_train, s_train = ajusteDados2(X_train)
    X_val, s_val = ajusteDados2(X_val)
    X_test, s_test = ajusteDados2(X_test)

    # y_train = nn.functional.one_hot(torch.tensor(y_train), num_classes=8).to(torch.float32).to(device)
    # y_val = nn.functional.one_hot(torch.tensor(y_val), num_classes=8).to(torch.float32).to(device)
    # y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()
    s_train = torch.tensor(s_train, dtype=torch.float32, device=device).detach()
    X_val = torch.tensor(X_val, dtype=torch.float32, device=device).detach()
    s_val = torch.tensor(s_val, dtype=torch.float32, device=device).detach()
    X_test = torch.tensor(X_test, dtype=torch.float32, device=device).detach()
    s_test = torch.tensor(s_test, dtype=torch.float32, device=device).detach()

    # encoder = ChangEncoder2().to(device)
    # regress = Regressor().to(device)
    model = MLP2().to(device)
    loss_fn = nn.MSELoss()
    # optimizer = optim.Adam(list(encoder.parameters()) + list(regress.parameters()), lr=0.001)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    n_epochs = 10   # number of epochs to run
    batch_size = 250  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
    trainLoss = []
    valLoss = []

    # training loop
    for epoch in range(n_epochs):
        # encoder.train()
        # regress.train()
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size,:,:]
                y_batch = s_train[start:start+batch_size,:]
                # forward pass
                # y_pred = regress(encoder(X_batch))
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                bar.set_postfix(loss=float(loss))
        # evaluate accuracy at end of each epoch
        # encoder.eval()
        # regress.eval()
        model.eval()
        # y_pred = regress(encoder(X_val))
        y_pred = model(X_val)
        valLoss.append(float(loss_fn(y_pred, s_val)))
        # y_pred = regress(encoder(X_train))
        # trainLoss.append(float(loss_fn(y_pred, s_train)))

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
    # fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
    fig.show()
    return model #encoder, regress


In [47]:
model = treinoModelo2(D[3])

Epoch 9: 100%|██████████| 1691/1691 [00:10<00:00, 157.67batch/s, loss=0.00262] 


In [48]:
PATH = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\reprodução chang\\modelos\\'
model.eval()
torch.save(model.state_dict(), PATH+'preditMLP2.pth')

In [13]:
PATH = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\reprodução chang\\modelos\\'
model = MLP().to(device)
model.load_state_dict(torch.load(PATH+'preditMLP.pth'))

<All keys matched successfully>

In [51]:
X, y = ajusteDados2(D[3][0,:1200].reshape(1,2,600) / 50)
X = torch.tensor(X, dtype=torch.float32, device=device).detach()

In [52]:
y_pred = model(X)

In [28]:
px.line(D[0][0,150:600])

In [53]:
px.line(y-y_pred.cpu().detach().numpy())

In [54]:
fig = px.line(y)
fig.add_trace(go.Scatter(y=y_pred.cpu().detach().numpy()[:,0], mode='lines', showlegend=True))
fig.add_trace(go.Scatter(y=y_pred.cpu().detach().numpy()[:,1], mode='lines', showlegend=True))

In [18]:
def treinoModelo3(dados):
    N = dados.shape[0]
    X = dados[:,:1200].reshape(N,2,600) / 50   # Normalizando para a faixa de -1 a 1 aprox.
    y = dados[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
    
    X_train, s_train = ajusteDados(X_train)
    X_val, s_val = ajusteDados(X_val)
    X_test, s_test = ajusteDados(X_test)

    # y_train = nn.functional.one_hot(torch.tensor(y_train), num_classes=8).to(torch.float32).to(device)
    # y_val = nn.functional.one_hot(torch.tensor(y_val), num_classes=8).to(torch.float32).to(device)
    # y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()
    s_train = torch.tensor(s_train, dtype=torch.float32, device=device).detach()
    X_val = torch.tensor(X_val, dtype=torch.float32, device=device).detach()
    s_val = torch.tensor(s_val, dtype=torch.float32, device=device).detach()
    X_test = torch.tensor(X_test, dtype=torch.float32, device=device).detach()
    s_test = torch.tensor(s_test, dtype=torch.float32, device=device).detach()

    encoder = ChangEncoder2().to(device)
    regress = Regressor().to(device)
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(list(encoder.parameters()) + list(regress.parameters()), lr=0.001)
    n_epochs = 10   # number of epochs to run
    batch_size = 250  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
    trainLoss = []
    valLoss = []

    # training loop
    for epoch in range(n_epochs):
        encoder.train()
        regress.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size,:,:]
                y_batch = s_train[start:start+batch_size,:]
                # forward pass
                y_pred = regress(encoder(X_batch))
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                bar.set_postfix(loss=float(loss))
        # evaluate accuracy at end of each epoch
        encoder.eval()
        regress.eval()
        y_pred = regress(encoder(X_val))
        valLoss.append(float(loss_fn(y_pred, s_val)))
        # y_pred = regress(encoder(X_train))
        # trainLoss.append(float(loss_fn(y_pred, s_train)))

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
    # fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
    fig.show()
    return encoder, regress


In [21]:
encoder, regress = treinoModelo3(D[3])

Epoch 9: 100%|██████████| 1294/1294 [02:31<00:00,  8.56batch/s, loss=0.0547]


In [22]:
PATH = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\reprodução chang\\modelos\\'
encoder.eval()
regress.eval()
torch.save(encoder.state_dict(), PATH+'preditEnc.pth')
torch.save(regress.state_dict(), PATH+'preditReg.pth')

In [19]:
encoder = ChangEncoder2()
regress = Regressor()
encoder.load_state_dict(torch.load(PATH+'preditEnc.pth'))
regress.load_state_dict(torch.load(PATH+'preditReg.pth'))

<All keys matched successfully>

In [20]:
def predlin(dados, ordem):
    N = dados.shape[0]
    X = dados[:,:1200].reshape(N,2,600) / 50   # Normalizando para a faixa de -1 a 1 aprox.
    y = dados[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
    
    X_train, s_train = ajusteDados(X_train)
    X_val, s_val = ajusteDados(X_val)
    X_test, s_test = ajusteDados(X_test)
    
    aux = np.linalg.inv(np.matmul(X_train[:,0,-ordem:].transpose(), X_train[:,0,-ordem:]))
    w1 = np.matmul(np.matmul(aux, X_train[:,0,-ordem:].transpose()), s_train[:,0])
    
    aux = np.linalg.inv(np.matmul(X_train[:,1,-ordem:].transpose(), X_train[:,1,-ordem:]))
    w2 = np.matmul(np.matmul(aux, X_train[:,1,-ordem:].transpose()), s_train[:,1])

    return w1, w2

In [23]:
ordem = 12
w1, w2 = predlin(D[3], ordem)

In [29]:
X, y = ajusteDados(D[3][0,:1200].reshape(1,2,600) / 50)
y_pred = np.vstack((np.matmul(X[:,0,-ordem:], w1), np.matmul(X[:,1,-ordem:], w2))).transpose()

In [30]:
fig = px.line(y)
fig.add_trace(go.Scatter(y=y_pred[:,0], mode='lines', showlegend=True))
fig.add_trace(go.Scatter(y=y_pred[:,1], mode='lines', showlegend=True))

In [34]:
px.line(D[2][0,150:600])

In [31]:
px.line(y-y_pred)

# Experimentações